# Hard Mode

Analysis by [Vincent Tjeng](https://vtjeng.com)

[Wordle](https://www.powerlanguage.co.uk/wordle/) has a game mode known as "hard mode", in which any guess must be consistent with the hints provided from previous guesses. While it can take more turns for _humans_ to solve Wordle in hard mode, it's actually more straightforward for a _computer_ to generate a solution for Wordle (and prove that it is optimal) via brute force in hard mode, since the pool of guesses allowed diminishes rapidly after a few turns

In [1]:
include("utils.jl");
ProgressMeter.ijulia_behavior(:clear);

Unable to init server: Could not connect: Connection refused
Unable to init server: Could not connect: Connection refused

(.:2031): Gdk-CRITICAL **: 13:04:33.074: gdk_cursor_new_for_display: assertion 'GDK_IS_DISPLAY (display)' failed

(.:2031): Gdk-CRITICAL **: 13:04:33.084: gdk_cursor_new_for_display: assertion 'GDK_IS_DISPLAY (display)' failed


In [2]:
cache_word_scores(SOLUTION_WORDS, SOLUTION_WORDS)

  2.441358 seconds (29.79 M allocations: 2.520 GiB, 18.06% gc time, 0.23% compilation time)


# Algorithm

The `get_optimal_strategy_exhaustive` function searches for the optimal strategy for Wordle in hard mode given a limited `turns_budget`. It either returns FAIL (if no strategy satisfying the `turns_budget` exists) or returns a strategy with the best worst-case number of turns (ties are broken by the average number of turns).

## Concepts
- We think of each guess as "splitting" the solution pool into one or more "shards". (A "shard" contains all of the words in the solution pool that would have returned the same response to the guess; the shards cover the solution pool but don't overlap). 
- For a particular solution pool, a strategy satisfying `turns_budget` exists if and only if there exists an initial guess that splits the solution pool into shards, where _each_ shard has a strategy that succeeds in at most `turns_budget - 1` turns.

## Optimizations
A naive algorithm tries every possible combination of guesses. We implement the following optimizations:

- We reduce `turns_budget` to the worst-case number of turns seen so far.
- For a given first guess, we try to find a successful strategy for the shards in decreasing order of size. We are more likely to fail to find a strategy for larger shards; if we do fail, we can short-circuit the computation and return FAIL for that first guess (and continue trying other guesses).
- When left with a budget of 1 turn and a solution pool that contains 2 or more words, we return FAIL.

### Considered but Unimplemented
- Since only 243-5=238 distinct responses are possible (three possibilities for hints for each letter, less the 5 combinations of exactly 1 yellow and 4 greens), we could return FAIL when left with a budget of 2 turns and a solution pool containing 239 or more words. _Implemented, but did not have a significant performance impact._
- We are currently trying the words in the order they were found in the word list. Sorting them by a heuristic might allow us to reduce `turns_budget` more quickly and stop trying guesses in less promising situations

# Hard mode, only guessing words that can be solutions, `turns_budget` = 4

No strategy exists when you constrain yourself to guess only words that can be solutions, given a budget of 4 turns.

In [3]:
r = get_optimal_strategy_exhaustive(
    SOLUTION_WORD_IDXS, 
    SOLUTION_WORD_IDXS, 
    hard_mode=true, 
    turns_budget=4,
    show_progress=true
)

Progress: 100%|█████████████████████████████████████████| Time: 0:00:04
  best_guess:              N/A
  best_max_num_turns:      N/A
  best_average_num_turns:  N/A
  num_skipped:             2315
  valid_guesses:           Tuple{String, Float64, Float64}[]


# Hard mode, only guessing words that can be solutions, `turns_budget` = 5

Two initial guesses succeed with a budget of 5 turns: "scowl" and "stamp". Excitingly, the brute-force search takes only 3 minutes!

In [4]:
r = get_optimal_strategy_exhaustive(
    SOLUTION_WORD_IDXS, 
    SOLUTION_WORD_IDXS, 
    hard_mode=true, 
    turns_budget=5,
    show_progress=true
)
num_turns, guess, strat = r;

Progress: 100%|█████████████████████████████████████████| Time: 0:03:05
  best_guess:              scamp
  best_max_num_turns:      5
  best_average_num_turns:  3.7161987041036717
  num_skipped:             2313
  valid_guesses:           [("scowl", 5.0, 3.752915766738661), ("scamp", 5.0, 3.7161987041036717)]


In [ ]:
plot_num_turns(
    num_turns, 
    title_text="5-turn solution for hard mode, starting with 'scamp'",
    saved_filename="strat_using_solutions_only_hard_mode_optimal.png"
)

In [6]:
open("hard_mode_strategy.md", "w") do io
    println(io, get_strategy_text([guess], strat, print_prefix = true))
end

# Hard mode, guess any 5-letter word, `turns_budget` = 4

It takes a bit more time, but we can show that allowing guesses that use _any_ of the ~12k 5-letter words doesn't allow you to solve in 4 steps!

In [7]:
cache_word_scores(ALL_WORDS, ALL_WORDS)

 74.896415 seconds (996.38 M allocations: 80.041 GiB, 18.04% gc time)


In [ ]:
r = get_optimal_strategy_exhaustive(
    ALL_WORD_IDXS, 
    SOLUTION_WORD_IDXS, 
    hard_mode=true, 
    turns_budget=4,
    show_progress=true
)
num_turns, guess, strat = r

Progress:  24%|██████████                               |  ETA: 2:18:33
  best_guess:              N/A
  best_max_num_turns:      N/A
  best_average_num_turns:  N/A
  num_skipped:             3171
  valid_guesses:           Tuple{String, Float64, Float64}[]